In [1]:
! git clone https://github.com/kumarankit1996/Dataset.git

fatal: destination path 'Dataset' already exists and is not an empty directory.


In [2]:
! ls Dataset/HackerEarth/
! pip install pygeocoder
! pip install python-googlegeocoder
! pip install -U googlemaps

address.csv  hm_train.csv   train_file.csv
hm_test.csv  test_file.csv  weights_v1.h5
Requirement already up-to-date: googlemaps in /usr/local/lib/python3.6/dist-packages (3.0.2)


In [2]:
import numpy as np
import pandas as pd
import math
import io
from keras.layers import concatenate
import random
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout,UpSampling2D, Dense, MaxPooling2D, BatchNormalization, Input, Flatten, Lambda
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.layers import Dense
from keras import optimizers
import os
import math
from sklearn.utils import shuffle
import itertools
from keras.callbacks import ModelCheckpoint, EarlyStopping
#from google.colab import files
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing, model_selection
#from pygeocoder import Geocoder
#from googlegeocoder import GoogleGeocoder
#import googlemaps

Using TensorFlow backend.


In [3]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection u'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-t

True

In [0]:
# Using google key
geocoder = GoogleGeocoder("xxxx")

# Read Dataset

In [5]:
Train=pd.read_csv("train_file.csv")
Test=pd.read_csv("test_file.csv")
# Address stored after extracting from lat and long
#address=pd.read_csv("address.csv")

# Preprocessing the dataset

In [6]:
def clean_sentence(sentence):
    # extract the alphabet from sentences
    sentence = re.sub("[^A-Za-z]", " ", sentence)
    # changing the case of sentence to lower case
    sentence = sentence.lower()
    # tokenizing words in sentences
    sentence = word_tokenize(sentence)
    # Converting to bi grams
    #sentence=nltk.bigrams(sentence)
    # defined the word lemmatizer object
    lemma = WordNetLemmatizer()
    # deleting the stop words and finding the lemmatize words
    sentence = [lemma.lemmatize(word,wordnet.VERB) for word in sentence if word not in set(stopwords.words("english"))]
    # joining the words to form sentence
    sentence = " ".join(sentence)
    return sentence

In [7]:
# variables for address from lat and long
house, street, city, state,pin, country=[],[],[],[],[],[]

In [8]:
# preprocessing the Train and test empty columns
Train['Description']=Train['Description'].fillna('Empty description')
Test['Description']=Test['Description'].fillna('Empty description')
# Combining train and test set for creating bag of words
bigdata = pd.concat([Train['Description'], Test['Description']], ignore_index=True)

In [9]:
# cleaning the dataset
desc=[]
for i in range(len(bigdata)):
    o=clean_sentence(bigdata[i])
    desc.append(o)

In [10]:
# Counting the frequency of each words and creating bag of words
count_vectorizer = CountVectorizer()
des = count_vectorizer.fit_transform(desc).toarray()

In [11]:
des.shape

(55899, 10170)

In [12]:
#pin=address['pin'].fillna(0)

In [13]:
 # Converting the target into one hot vector and the features into categories
def category(data,s="value"):
      if s=="target":
        target=[]
        for i in range(len(data)):
            if data[i]=='SINGLE FAMILY / DUPLEX':
                target.append(0)
            elif data[i]=='COMMERCIAL':
                target.append(1)
            elif data[i]=='MULTIFAMILY':
                target.append(2)
            elif data[i]=='INSTITUTIONAL':
                target.append(3)
            elif data[i]=='INDUSTRIAL':
                target.append(4)
        target = to_categorical(target)
        return target
      else:
        # Transform categoryinto numerical values 
        encoder = LabelEncoder()
        encoder.fit(data)
        data = encoder.transform(data)
        print(data[:20])
        return data.reshape(-1,1)

In [14]:
# Extracting the features from lat and long
def location(lat,lon):
  global house, street, city, state,pin, country
  for i,j in zip(lat,lon):
    try:
      reverse = geocoder.get((i,j))
      address=[]
      address.append(str(reverse[1]))
      temp=address[0].split(', ')
      #house.append(temp[0].split(' ',1)[0])
      try:
        street.append(temp[0].split(' ',1)[1])
      except:
        street.append(np.nan)
      try:
        city.append(temp[1])
      except:
        city.append(np.nan)
      try:
        state.append(temp[2].split(' ')[0])
      except:
        state.append(np.nan)
      try:
        pin.append(temp[2].split(' ')[1])
      except:
        pin.append(np.nan)
      try:
        country.append(temp[3])
      except:
        country.append(np.nan)
    except:
      continue

In [14]:
# preprocessing the features from the dataset
def preprocess(data,s="train"):
  permit_type=data['Permit Type']
  permit_type=category(permit_type)
  #print(permit_type.shape)

  action_type=data['Action Type'].astype(str)#.str[:3]
  action_type=category(action_type)
  #print(action_type.shape)

  work_type=data['Work Type']
  work_type=category(work_type)
  #print(work_type.shape)

  status=data['Status'].astype(str)
  status=category(status)
  #print(status.shape)

  data['Master Use Permit']=np.where(data['Master Use Permit']>0,1,data['Master Use Permit'])
  master_permit=data['Master Use Permit'].fillna(0)
  master_permit=category(master_permit)
  #print(master_permit.shape)

  data['Latitude']=data['Latitude'].fillna(0)
  lat=data['Latitude'].astype(float)
  #lat=np.array(lat).reshape(-1,1)
  lat[0]

  data['Longitude']=data['Longitude'].fillna(0)
  lon=data['Longitude'].astype(float)
  #long=np.array(long).reshape(-1,1)
  lon[0]
  
  #location(lat,lon)
  #z=np.hstack([np.array(street).reshape(-1,1),np.array(city).reshape(-1,1),np.array(state).reshape(-1,1),np.array(pin).reshape(-1,1),np.array(country).reshape(-1,1)])
  #v=pd.DataFrame(z)
  #x=np.array(v['pin'])
  #pincode=np.array([int(i) if'98' in i else 0 for i in x]).reshape(-1,1)
  #print(pincode.shape)
    
    # Finding the difference between dates for better understanding of the model
  adate=data['Application Date'].fillna('1919-04-13')
  adate=pd.to_datetime(adate)
  idate=data['Issue Date'].fillna('2019-04-13')
  idate=pd.to_datetime(idate)
  issued_in=(idate-adate).astype(str)
  issued_in.tolist()
  issued_in=[int(i.split('days',1)[0]) for i in issued_in]
  issued_in=np.array(issued_in).reshape(-1,1)
  #print(issued_in.shape)

  idate=data['Issue Date'].fillna('1919-04-13')
  idate=pd.to_datetime(idate)
  fdate=data['Final Date'].fillna('2019-04-13')
  fdate=pd.to_datetime(fdate)
  inspected_in=(fdate-idate).astype(str)
  inspected_in.tolist()
  inspected_in=[int(j.split('days',1)[0]) for j in inspected_in]
  inspected_in=np.array(inspected_in).reshape(-1,1)
  #print(inspected_in.shape)

  fdate=data['Final Date'].fillna('1919-04-13')
  fdate=pd.to_datetime(fdate)
  edate=data['Expiration Date'].fillna('2019-04-13')
  edate=pd.to_datetime(edate)
  expire_in=(edate-idate).astype(str)
  expire_in.tolist()
  expire_in=[int(k.split('days',1)[0]) for k in expire_in]
  expire_in=np.array(expire_in).reshape(-1,1)
  #print(expire_in.shape)
  
    # combining the processed features
  if s=="train":
    X=np.hstack((permit_type,action_type,work_type,status,master_permit,issued_in,inspected_in,expire_in,des[:len(Train['Description'])]))
  else:
    X=np.hstack((permit_type,action_type,work_type,status,master_permit,issued_in,inspected_in,expire_in,des[len(Train['Description']):]))
  return X

In [15]:
train_data=preprocess(Train)
test_data=preprocess(Test,'test')

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0]
[ 0  0  0  4  0  0  0  1  0  0  0  8  0  8  8  0 15  0  0  0]
[0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1]
[10  8  8  6  8  6  6  8  6  6  8  9  6  8  6 10  0  8  6  3]
[0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0]
[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
[ 0  0  4  7  0 15  7  0  7  0  0  0  7  0  7  0  0  7  0  2]
[0 0 0 1 0 0 1 0 1 0 0 1 1 0 1 0 1 1 0 0]
[ 8  8 11 10  8  6  6  6  8  8  8  8  2  8 10 10 10  2  8  0]
[0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0]


In [16]:
train_data.shape,test_data.shape

((33539, 10178), (22360, 10178))

# Extracting Label

In [20]:
# Converting the targte into one hot vector
Train_label=Train['Category']
Y=category(Train_label,"target")
Y[:10]

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

# Neural Network Model

In [31]:
mc = ModelCheckpoint(filepath='weights_v5.h5', monitor='val_loss',period=1,save_best_only=True,save_weights_only=True,mode='auto', verbose = 3)
es = EarlyStopping(patience=1000, monitor='val_loss', min_delta=0.0005, mode='auto')

In [32]:
input_dim = len(train_data[0])

model = Sequential()
model.add(Dense(100, input_dim = input_dim , activation = 'relu'))
model.add(Dense(150, activation = 'relu'))
model.add(Dense(150, activation = 'relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer =optimizers.Adadelta(lr=0.01) , metrics = ['accuracy'] )

In [33]:
model.fit(train_data, Y, epochs = 200,callbacks=[mc,es], batch_size = 64,validation_split=0.2)

Train on 26831 samples, validate on 6708 samples
Epoch 1/200
26831/26831 [==============================] - 22s 818us/step - loss: 3.3265 - acc: 0.4856 - val_loss: 3.0623 - val_acc: 0.5343

Epoch 00001: val_loss improved from inf to 3.06225, saving model to weights_v5.h5
Epoch 2/200
26831/26831 [==============================] - 17s 649us/step - loss: 2.7666 - acc: 0.5431 - val_loss: 2.7176 - val_acc: 0.5361

Epoch 00002: val_loss improved from 3.06225 to 2.71760, saving model to weights_v5.h5
Epoch 3/200
26831/26831 [==============================] - 20s 745us/step - loss: 2.5936 - acc: 0.5500 - val_loss: 3.8565 - val_acc: 0.4529

Epoch 00003: val_loss did not improve from 2.71760
Epoch 4/200
26831/26831 [==============================] - 18s 664us/step - loss: 1.9970 - acc: 0.5455 - val_loss: 1.8752 - val_acc: 0.5701

Epoch 00004: val_loss improved from 2.71760 to 1.87518, saving model to weights_v5.h5
Epoch 5/200
26831/26831 [==============================] - 17s 639us/step - loss: 


Epoch 00082: val_loss did not improve from 0.76564
Epoch 83/200
26831/26831 [==============================] - 19s 715us/step - loss: 0.8243 - acc: 0.7246 - val_loss: 0.7938 - val_acc: 0.6832

Epoch 00083: val_loss did not improve from 0.76564
Epoch 84/200
26831/26831 [==============================] - 19s 720us/step - loss: 0.8126 - acc: 0.7261 - val_loss: 0.8452 - val_acc: 0.7361

Epoch 00084: val_loss did not improve from 0.76564
Epoch 85/200
26831/26831 [==============================] - 20s 746us/step - loss: 0.8043 - acc: 0.7284 - val_loss: 1.2429 - val_acc: 0.6667

Epoch 00085: val_loss did not improve from 0.76564
Epoch 86/200
26831/26831 [==============================] - 22s 809us/step - loss: 0.8107 - acc: 0.7274 - val_loss: 0.8365 - val_acc: 0.6427

Epoch 00086: val_loss did not improve from 0.76564
Epoch 87/200
26831/26831 [==============================] - 20s 760us/step - loss: 0.8095 - acc: 0.7259 - val_loss: 1.3505 - val_acc: 0.6304

Epoch 00087: val_loss did not impr


Epoch 00124: val_loss did not improve from 0.74435
Epoch 125/200
26831/26831 [==============================] - 21s 783us/step - loss: 0.7450 - acc: 0.7456 - val_loss: 0.9190 - val_acc: 0.6887

Epoch 00125: val_loss did not improve from 0.74435
Epoch 126/200
26831/26831 [==============================] - 23s 862us/step - loss: 0.7460 - acc: 0.7447 - val_loss: 0.8099 - val_acc: 0.7224

Epoch 00126: val_loss did not improve from 0.74435
Epoch 127/200
26831/26831 [==============================] - 19s 718us/step - loss: 0.7415 - acc: 0.7434 - val_loss: 1.0188 - val_acc: 0.7463

Epoch 00127: val_loss did not improve from 0.74435
Epoch 128/200
26831/26831 [==============================] - 21s 786us/step - loss: 0.7358 - acc: 0.7479 - val_loss: 1.0104 - val_acc: 0.7452

Epoch 00128: val_loss did not improve from 0.74435
Epoch 129/200
26831/26831 [==============================] - 23s 839us/step - loss: 0.7429 - acc: 0.7463 - val_loss: 0.8975 - val_acc: 0.6914

Epoch 00129: val_loss did not

In [0]:
#files.download('weights_v2.h5')

In [34]:
# Loading the best weights of the model
model.load_weights('weights_v5.h5')

# Prediction

In [35]:
predicted=model.predict(test_data)

In [36]:
p=np.argmax(predicted,axis=1)
predict=[]
for i in range(len(p)):
    if p[i]==0:
        predict.append('SINGLE FAMILY / DUPLEX')
    elif p[i]==1:
        predict.append('COMMERCIAL')
    elif p[i]==2:
        predict.append('MULTIFAMILY')
    elif p[i]==3:
        predict.append('INSTITUTIONAL')
    elif p[i]==4:
        predict.append('INDUSTRIAL')

In [37]:
predict=np.array(predict).reshape(-1,1)
uid=np.array(Test['Application/Permit Number']).reshape(-1,1)
output=np.hstack((uid,predict))

In [39]:
output=pd.DataFrame(output)
output.columns=['Application/Permit Number','Category']
output.to_csv("output11.csv",sep=",",index=False)
#files.download("output1.csv")

# SVM Model

In [20]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(train_data,np.argmax(Y,axis=1),test_size = 0.1, random_state = 0)

In [ ]:
from sklearn.svm import SVC
# defining the SVM Model
SVM_model=SVC(kernel='rbf',C=2,gamma=0.05)
history2=SVM_model.fit(train_x,train_y)

In [ ]:
#Calculate Erms 
def ERMS(Predicted,Actual):
    total=0.0
    counter=0
    for i in range(0,len(Predicted)):
        total+=math.pow((Actual[i]-Predicted[i]),2)
        #count for match
        if (int(np.around(Predicted[i])) == Actual[i]):
            counter+=1
    Accuracy=(counter*100/len(Predicted))
    #print(counter)
    Erms=math.sqrt(total/len(Predicted))
    return Erms, Accuracy

In [ ]:
SVM_PredictedTarget=SVM_model.predict(test_x)
Erms, SVM_Accu=ERMS(SVM_PredictedTarget,test_y)
print("Erms:",Erms)
print("SVM Val Accu:",SVM_Accu)

# Prediction

In [0]:
SVM_PredictedTarget=SVM_model.predict(test_data)

In [ ]:
p=SVM_PredictedTarget
predict=[]
for i in range(len(p)):
    if p[i]==0:
        predict.append('SINGLE FAMILY / DUPLEX')
    elif p[i]==1:
        predict.append('COMMERCIAL')
    elif p[i]==2:
        predict.append('MULTIFAMILY')
    elif p[i]==3:
        predict.append('INSTITUTIONAL')
    elif p[i]==4:
        predict.append('INDUSTRIAL')

In [ ]:
predict=np.array(predict).reshape(-1,1)
uid=np.array(Test['Application/Permit Number']).reshape(-1,1)
output=np.hstack((uid,predict))

In [ ]:
output=pd.DataFrame(output)
output.columns=['Application/Permit Number','Category']
output.to_csv("svmoutput1.csv",sep=",",index=False)